In [1]:
# RDKit
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem.MolStandardize import rdMolStandardize

# Utilitários
import os
import re
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
from IPython.display import clear_output

# Transformers
import torch
from transformers import AutoTokenizer, AutoModel

# Scikit-learn
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import NuSVC
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

# XGBoost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Imports base
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, HistGradientBoostingClassifier
)
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

# Modelos externos
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Data Set Both

# Limpeza de Dados

In [2]:
from rdkit import Chem

RDLogger.DisableLog('rdApp.info')   # silencia mensagens informativas

def smiles_valido(smiles):
    """
    Verifica se um SMILES é válido.
    
    Parâmetros:
        smiles (str): Representação SMILES.
    
    Retorna:
        bool: True se o SMILES é válido, False caso contrário.
    """
    if pd.isna(smiles) or not isinstance(smiles, str) or smiles.strip() == "":
        return False
    
    smiles = smiles.strip()
    
    try:
        # Primeiro tenta com sanitização normal
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
        if mol is not None:
            return True
    except Exception:
        pass  # Ignora e tenta de novo sem sanitização
    
    try:
        # Se falhar, tenta sem sanitização e sanitiza manualmente
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        if mol is not None:
            Chem.SanitizeMol(mol, catchErrors=True)
            return True
    except Exception:
        pass
    
    return False

# Descritores + Estrutura de alerta

In [3]:
# Disable informational RDKit logs
RDLogger.DisableLog('rdApp.info')

# Preload RDKit uncharger object for performance
_uncharger = rdMolStandardize.Uncharger()

# --- Molecule Neutralization ---
def neutralize_molecule(mol):
    """
    Neutralizes a molecule using RDKit's standardization tools.

    Parameters:
        mol (rdkit.Chem.Mol): RDKit molecule object.

    Returns:
        rdkit.Chem.Mol or None: Neutralized molecule, or original molecule if neutralization fails.
    """
    if mol is None:
        return None
    try:
        # Cleanup handles salting, normalization, and common charges
        mol = rdMolStandardize.Cleanup(mol)
        mol = _uncharger.Uncharge(mol)
        return mol
    except Exception:
        return mol  # Return original molecule if neutralization fails


# --- SMILES to Molecule Conversion ---
def smiles_to_mol(smiles, neutralize=True):
    """
    Converts a SMILES string to an RDKit molecule object, with optional neutralization.

    Parameters:
        smiles (str): SMILES string.
        neutralize (bool): Whether to neutralize the molecule after parsing.

    Returns:
        rdkit.Chem.Mol or None: RDKit molecule object, or None if parsing fails.
    """
    if pd.isna(smiles) or smiles.strip() == "":
        return None

    try:
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
    except Exception:
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        if mol is not None:
            Chem.SanitizeMol(mol, catchErrors=True)

    if neutralize:
        mol = neutralize_molecule(mol)

    return mol


# --- Precompiled Descriptor List ---
_DESC_FUNCS = [(name, fn) for name, fn in Descriptors.descList]

def compute_descriptors(mol):
    """
    Computes molecular descriptors for a given RDKit molecule.

    Parameters:
        mol (rdkit.Chem.Mol): RDKit molecule object.

    Returns:
        dict: Dictionary of descriptor names and values, NaN if calculation fails.
    """
    if mol is None:
        return {f"desc_{name}": np.nan for name, _ in _DESC_FUNCS}

    out = {}
    for name, fn in _DESC_FUNCS:
        try:
            out[f"desc_{name}"] = fn(mol)
        except Exception:
            out[f"desc_{name}"] = np.nan
    return out


# --- Substructure Search and Descriptor Calculation ---
def check_substructures_and_descriptors(
    df,
    df_structures,
    smiles_col='SMILES',
    structure_smiles_col='SMILES',
    neutralize=True,
    use_smarts=True
):
    """
    Checks for the presence of substructures in compounds and computes molecular descriptors.

    Parameters:
        df (pd.DataFrame): DataFrame containing compounds with a SMILES column.
        df_structures (pd.DataFrame): DataFrame containing substructures (as SMILES or SMARTS).
        smiles_col (str): Name of the SMILES column in df.
        structure_smiles_col (str): Name of the SMILES/SMARTS column in df_structures.
        neutralize (bool): Whether to neutralize compounds before comparison.
        use_smarts (bool): If True, interpret df_structures[structure_smiles_col] as SMARTS.

    Returns:
        pd.DataFrame: Original DataFrame + descriptor columns (prefix "desc_") + binary substructure columns (prefix "sub_").
    """

    # Clean column names in the substructure DataFrame
    df_structures = df_structures.copy()
    df_structures.columns = df_structures.columns.str.strip()

    # Prepare substructure patterns
    patterns = {}
    for pat in df_structures[structure_smiles_col].dropna().astype(str).str.strip().unique():
        if pat == "":
            continue
        try:
            mol_pat = Chem.MolFromSmarts(pat) if use_smarts else Chem.MolFromSmiles(pat)
            if mol_pat is not None:
                # Column name for this pattern
                colname = f"sub_{pat}"
                # Avoid overly long column names
                if len(colname) > 60:
                    colname = f"sub_{hash(pat)}"
                patterns[colname] = mol_pat
        except Exception:
            continue

    # Prepare storage for results
    substructure_results = {col: [] for col in patterns.keys()}
    descriptor_results = {f"desc_{name}": [] for name, _ in _DESC_FUNCS}

    # Process each compound
    for smiles in tqdm(df[smiles_col], desc="Processing molecules", unit="mol"):
        mol = smiles_to_mol(smiles, neutralize=neutralize)

        # Check substructures
        for colname, patt in patterns.items():
            hit = 0
            if mol is not None:
                try:
                    hit = int(mol.HasSubstructMatch(patt))
                except Exception:
                    hit = 0
            substructure_results[colname].append(hit)

        # Compute descriptors
        descs = compute_descriptors(mol)
        for k in descriptor_results.keys():
            descriptor_results[k].append(descs.get(k, np.nan))

    # Build DataFrames for substructures and descriptors
    df_subs = pd.DataFrame(substructure_results, index=df.index) if substructure_results else pd.DataFrame(index=df.index)
    df_descs = pd.DataFrame(descriptor_results, index=df.index)

    # Concatenate results with the original DataFrame (keeping index)
    df_final = pd.concat([df.reset_index(drop=False), df_descs, df_subs], axis=1)

    return df_final

In [4]:
modelo = "DeepChem/ChemBERTa-77M-MTR"

# Carregar o tokenizer e o modelo ChemBERTa
tokenizer = AutoTokenizer.from_pretrained(modelo,trust_remote_code=True)
model = AutoModel.from_pretrained(modelo, trust_remote_code=True)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/

In [5]:
# (3) Função para obter embedding de um SMILES
def get_embedding(smiles):
    inputs = tokenizer(smiles, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze()  # Remove batch dimension
    return embedding.cpu().numpy()  # Convertendo para numpy array para facilitar

# Classificação

In [6]:
df_vivo= pd.read_csv('in vivo + cpdb.csv')
df_vivo.shape

(19883, 9)

In [7]:
df_vivo['SMILES_valido'] = df_vivo['SMILES'].apply(smiles_valido)
df_vivo['SMILES_valido'].value_counts()

[22:30:43] WARNING: not removing hydrogen atom without neighbors
[22:30:43] WARNING: not removing hydrogen atom without neighbors


SMILES_valido
True    19883
Name: count, dtype: int64

In [8]:
# Drop unnecessary columns
df_vivo.drop(columns=['Chemical', 'Identificador', 'SMILES_valido','species','strain','Male','Female'], inplace=True)

# Convert text columns to lowercase
df_vivo['Results'] = df_vivo['Results'].str.lower()
df_vivo['Type'] = df_vivo['Type'].str.lower()

print(df_vivo.shape)

# Remove duplicate rows if any
df_vivo.drop_duplicates(inplace=True)

df_vivo.shape

(19883, 3)


(3617, 3)

In [9]:
# Identify SMILES (in the filtered DataFrame) that have more than one 'Results' value
smiles_multiple_results = df_vivo.groupby("SMILES")["Results"].nunique()
smiles_multiple_results = smiles_multiple_results[smiles_multiple_results > 1].index

# Remove SMILES that have more than one result
df_final_vivo = df_vivo[~df_vivo["SMILES"].isin(smiles_multiple_results)]

print("Total de compostos não divergentes:", len(df_final_vivo))

df_organicos = df_final_vivo[df_final_vivo["SMILES"].str.contains(r"C(?![a-z])", regex=True, na=False)]

print("Total de compostos orgânicos:", len(df_organicos))

# Drop 'Type' column as it is no longer needed
df_final_vivo = df_organicos.drop(columns='Type')

# Reset index for clean ordering
df_final_vivo.reset_index(drop=True, inplace=True)

# Show final shape
df_final_vivo.shape

Total de compostos não divergentes: 2227
Total de compostos orgânicos: 2090


(2090, 2)

In [10]:
# garante que a coluna exista
assert "SMILES" in df_final_vivo.columns, "Coluna 'SMILES' não encontrada no dataframe"

# conta quantos SMILES não têm 'C' (maiúsculo)
sem_carbono = df_final_vivo[~df_final_vivo["SMILES"].str.contains("C", na=False)]

print("Número de SMILES sem carbono:", len(sem_carbono))


Número de SMILES sem carbono: 0


In [11]:
df_structures = pd.read_csv('Estruturas de alerta.csv')

In [12]:
# Exemplo de uso
df_descritores_vivo = check_substructures_and_descriptors(df_final_vivo, df_structures)
df_descritores_vivo.shape

Processing molecules: 100%|██████████| 2090/2090 [00:10<00:00, 204.55mol/s]


(2090, 378)

In [13]:
df_descritores_vivo["Results"].value_counts()

Results
negative    1286
positive     804
Name: count, dtype: int64

In [14]:
smiles_list = df_final_vivo['SMILES'].tolist()
len(smiles_list)

# Apply the function with a progress bar
embeddings = [get_embedding(smiles) for smiles in tqdm(smiles_list, desc="Generating embeddings")]
embeddings = pd.DataFrame(np.vstack(embeddings))

Generating embeddings: 100%|██████████| 2090/2090 [00:03<00:00, 664.93it/s]


In [15]:
df_final = pd.concat([embeddings.reset_index(drop=True),
                      df_descritores_vivo.reset_index(drop=True)], axis=1)

In [16]:
X = df_final.drop(columns=['SMILES', 'Results']).fillna(0)
X.columns = X.columns.astype(str)   # <- garante que todas são string
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

y = df_final['Results'].astype(str).str.lower().map({'positive':1, 'negative':0})

In [17]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# =========================
# Model Catalog
# =========================
model_zoo = {
    # Linear regressions / linear classifiers
    "LogisticRegression": LogisticRegression(max_iter=1000, n_jobs=-1),
    "RidgeClassifier": RidgeClassifier(),
    "SGDClassifier": SGDClassifier(max_iter=1000, tol=1e-3, n_jobs=-1),
    "PassiveAggressive": PassiveAggressiveClassifier(max_iter=1000, random_state=42),
    "Perceptron": Perceptron(max_iter=1000, tol=1e-3, random_state=42),

    # Support Vector Machines (SVM)
    "LinearSVC": LinearSVC(),
    "SVC": SVC(),
    "NuSVC": NuSVC(),

    # Nearest neighbors
    "KNeighbors": KNeighborsClassifier(),

    # Decision trees and tree ensembles
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "ExtraTree": ExtraTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),

    # Other ensemble methods
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42, n_jobs=-1),

    # Naive Bayes
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),

    # Discriminant analysis
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),

    # Simple neural networks
    "MLPClassifier": MLPClassifier(max_iter=1000, random_state=42),

    # External gradient boosting models
    "XGBoost": XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        use_label_encoder=False,
        eval_metric="logloss",
        n_jobs=-1
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    "CatBoost": CatBoostClassifier(
        iterations=500,
        learning_rate=0.05,
        depth=6,
        verbose=False,
        random_state=42
    )
}

# =========================
# BALANCEADORES
# =========================
balancers = {
    "original": None,  # sem sampling
    "under": RandomUnderSampler(random_state=42),
    "over": RandomOverSampler(random_state=42),
    "smote": SMOTE(random_state=42),
}

# =========================
# PRÉ-PROCESSAMENTO DO X E y
# =========================
X = df_final.drop(columns=['SMILES', 'Results']).fillna(0)

# 🔥 Correção obrigatória
X.columns = X.columns.astype(str)

y = (
    df_final['Results']
    .astype(str)
    .str.lower()
    .map({'positive': 1, 'negative': 0})
)

# =========================
# CROSS-VALIDATION E MÉTRICAS
# =========================
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1": "f1",
    "roc_auc": "roc_auc",
    "precision": "precision",
    "recall": "recall",
}

# garantir pasta
import os
os.makedirs("Resultados", exist_ok=True)
os.makedirs("Resultados_folds", exist_ok=True)

# =========================
# LOOP PRINCIPAL: sampling → modelos → resultados
# =========================
for balance_name, sampler in balancers.items():

    print(f"\n=== Rodando balanceamento: {balance_name} ===")

    pipelines = {}

    # Construção dos pipelines
    for name, clf in model_zoo.items():

        if sampler is None:
            pipe = Pipeline(steps=[
                ("scaler", StandardScaler()),
                ("clf", clf)
            ])
        else:
            pipe = ImbPipeline(steps=[
                ("scaler", StandardScaler()),
                ("sampler", sampler),
                ("clf", clf)
            ])

        pipelines[name] = pipe

    # =====================
    # 1) MÉDIAS E STD
    # =====================
    summary_rows = []

    # =====================
    # 2) VALORES POR FOLD
    # =====================
    fold_rows = []

    for name, pipe in pipelines.items():
        cvres = cross_validate(
            pipe, X, y,
            cv=cv,
            scoring=scoring,
            n_jobs=-1,
            return_train_score=False
        )

        # ---------- salvar *fold a fold* ----------
        for i in range(cv.get_n_splits()):
            fold_rows.append({
                "Model": name,
                "Fold": i + 1,
                "Accuracy": cvres["test_accuracy"][i],
                "BalancedAcc": cvres["test_balanced_accuracy"][i],
                "F1": cvres["test_f1"][i],
                "ROC_AUC": cvres["test_roc_auc"][i],
                "Precision": cvres["test_precision"][i],
                "Recall": cvres["test_recall"][i],
                "FitTime": cvres["fit_time"][i],
                "ScoreTime": cvres["score_time"][i],
            })

        # ---------- salvar *média e desvio padrão* ----------
        summary_rows.append({
            "Model": name,
            "Accuracy_mean": np.mean(cvres["test_accuracy"]),
            "Accuracy_std":  np.std(cvres["test_accuracy"], ddof=1),
            "BalAcc_mean":   np.mean(cvres["test_balanced_accuracy"]),
            "BalAcc_std":    np.std(cvres["test_balanced_accuracy"], ddof=1),
            "F1_mean":       np.mean(cvres["test_f1"]),
            "F1_std":        np.std(cvres["test_f1"], ddof=1),
            "ROC_AUC_mean":  np.mean(cvres["test_roc_auc"]),
            "ROC_AUC_std":   np.std(cvres["test_roc_auc"], ddof=1),
            "Precision_mean":np.mean(cvres["test_precision"]),
            "Precision_std": np.std(cvres["test_precision"], ddof=1),
            "Recall_mean":   np.mean(cvres["test_recall"]),
            "Recall_std":    np.std(cvres["test_recall"], ddof=1),
            "FitTime_mean":  np.mean(cvres["fit_time"]),
            "FitTime_std":   np.std(cvres["fit_time"], ddof=1),
            "ScoreTime_mean":np.mean(cvres["score_time"]),
            "ScoreTime_std": np.std(cvres["score_time"], ddof=1),
        })

    # ============================
    # Salvar RESUMO (médias/std)
    # ============================
    final_summary = (
        pd.DataFrame(summary_rows)
        .sort_values(by="F1_mean", ascending=False)
        .reset_index(drop=True)
    )

    path_summary = f"Resultados/Embdest_{balance_name}.csv"
    final_summary.to_csv(path_summary, index=False)

    print(f"[OK] Salvo resumo: {path_summary}")
    print(final_summary.head(5))

    # ============================
    # Salvar FOLDS (raw)
    # ============================
    df_folds = pd.DataFrame(fold_rows)
    path_folds = f"Resultados_folds/Embdest_{balance_name}.csv"
    df_folds.to_csv(path_folds, index=False)

    print(f"[OK] Salvo folds: {path_folds}")



=== Rodando balanceamento: original ===
[OK] Salvo resumo: Resultados/Embdest_original.csv
           Model  Accuracy_mean  Accuracy_std  BalAcc_mean  BalAcc_std  \
0       LightGBM       0.803349      0.017537     0.779197    0.021826   
1        XGBoost       0.802871      0.019112     0.777142    0.022687   
2       CatBoost       0.804306      0.026635     0.776938    0.029974   
3  MLPClassifier       0.790431      0.029564     0.773583    0.032450   
4     ExtraTrees       0.807177      0.029499     0.775961    0.032132   

    F1_mean    F1_std  ROC_AUC_mean  ROC_AUC_std  Precision_mean  \
0  0.724406  0.029769      0.859892     0.012313        0.785647   
1  0.721442  0.030729      0.865855     0.018496        0.790204   
2  0.720457  0.040736      0.860716     0.018664        0.800710   
3  0.719349  0.041240      0.838142     0.020254        0.743068   
4  0.718402  0.044656      0.853345     0.019772        0.819859   

   Precision_std  Recall_mean  Recall_std  FitTime_mea